In [ ]:
# expose parent directory to import modules
import os
import sys

ROOT_DIR = os.getcwd()
while os.path.basename(ROOT_DIR) != 'DatasetsStatistics':
    ROOT_DIR = os.path.abspath(os.path.join(ROOT_DIR,'..'))
sys.path.insert(0,ROOT_DIR)
os.chdir(ROOT_DIR)

TASK = 'segmentation'

In [ ]:
import json
import random
import cv2
import numpy as np

from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

from pycocotools import mask as maskUtils

import pathlib.Path as Path


In [ ]:
def myAnnToRLE(h, w, segm):
    if type(segm) == list:
        # polygon -- a single object might consist of multiple parts
        # we merge all parts into one mask rle code
        rles = maskUtils.frPyObjects(segm, h, w)
        rle = maskUtils.merge(rles)
    elif type(segm['counts']) == list:
        # uncompressed RLE
        rle = maskUtils.frPyObjects(segm, h, w)
    else:
        # rle
        rle = segm
    return rle


In [ ]:

def myAnnToMask(h, w, segm):
    """
    Convert annotation which can be polygons, uncompressed RLE, or RLE to binary mask.
    :return: binary mask (numpy 2D array)
    """
    rle = myAnnToRLE(h, w, segm)
    m = maskUtils.decode(rle)
    return m


In [ ]:

def load_annotations(path, file_name):
    with open(os.path.join(path, file_name), 'rb') as f:
        data = json.load(f)
    return data



In [ ]:

def get_video_id_to_video_dict(data):
    video_id_to_video = {}

    for vid in data['videos']:
        id = vid['id']
        video_id_to_video[id] = vid

    return video_id_to_video


In [ ]:


def get_annotations_per_video(data):

    anns_per_vid = {}

    for ann in data['annotations']:
        vid_id = ann['video_id']

        if vid_id not in anns_per_vid:
            anns_per_vid[vid_id] = []

        anns_per_vid[vid_id].append(ann)

    return anns_per_vid



In [ ]:

def process_video(video, anns, images_path):

    colormap = {}

    for inst in anns:
        id = inst['id']
        colormap[id] = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))

    N = len(video['file_names'])

    width = video['width']
    height = video['height']
    video_name = video['file_names'][0].split('/')[0] + ".mp4"

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    videoWriter = cv2.VideoWriter(video_name, fourcc, 10, (width, height))

    for frame_idx in range(N):
        frame_name = video['file_names'][frame_idx]

        img = cv2.imread(os.path.join(images_path, frame_name))



        mask = np.zeros((height,width,3), np.uint8)



        for inst in anns:
            bbox = inst['bboxes'][frame_idx]
            id = inst['id']

            segm = inst['segmentations'][frame_idx]



            if bbox == None:
                continue

            x1 = int(bbox[0])
            y1 = int(bbox[1])

            x2 = x1 + int(bbox[2])
            y2 = y1 + int(bbox[3])

            start_point = (x1, y1)
            end_point = (x2, y2)
            color = colormap[id]
            thickness = 2

            # img = cv2.rectangle(img, start_point, end_point, color, thickness)

            inst_mask = myAnnToMask(height, width, segm)

            inst_mask_img = np.zeros((height,width,3), np.uint8)

            inst_mask_img[:, :, 0] = inst_mask * color[0]
            inst_mask_img[:, :, 1] = inst_mask * color[1]
            inst_mask_img[:, :, 2] = inst_mask * color[2]

            mask += inst_mask_img


        img = cv2.addWeighted(img, 0.5, mask, 0.5, 0.0)

        folder_name = frame_name.split('/')[0]
        file_name = frame_name.split('/')[1]
        folder_path = os.path.join("./out_frames", folder_name)
        if not os.path.exists(folder_path):
            os.mkdir(folder_path)
        cv2.imwrite(os.path.join(folder_path, file_name), img)

        videoWriter.write(img)

    videoWriter.release()



In [ ]:

def parse_data(path, file_name):
    anns = load_annotations(path, file_name)
    videos = get_video_id_to_video_dict(anns)
    anns_per_vid = get_annotations_per_video(anns)

    return videos, anns_per_vid



In [ ]:
data_path = Path("data") / "ytvis"
images_path = data_path / "train"/ "JPEGImages"
file_name = 'train.json'

In [ ]:
videos, anns_per_vid = parse_data(data_path, file_name)

In [ ]:
for vid_id in videos:
    vid = videos[vid_id]
    vid_anns = anns_per_vid[vid_id]

    process_video(vid, vid_anns, images_path)
